In [12]:
import os
from os.path import join as pjoin
from copy import deepcopy

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

RS = 42
SHUFFLE = True

In [ ]:
train = pd.read_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025.csv"
)

In [ ]:
def stratified_k_fold(X, y, g, k):
    return StratifiedGroupKFold(n_splits=k, shuffle=SHUFFLE, random_state=RS).split(
        X, y, g
    )

In [ ]:
train["group"] = train["author"].str.lower()
train.loc[train["group"] == "unknown", "group"] = train.loc[train["group"] == "unknown", "filename"]

In [ ]:
split = list(stratified_k_fold(X=train, y=train["primary_label"], g=train["group"], k=5))

In [ ]:
all_unique_birds = set(train["primary_label"])
len(all_unique_birds)

In [ ]:
split = [list(el) for el in split]
for fold_i in range(len(split)):
    fold_train = train.iloc[split[fold_i][0]]
    missing_birds = all_unique_birds - set(fold_train["primary_label"])
    if missing_birds:
        print(f"Fold {fold_i} missing species: {missing_birds}")
        index_to_add = train.loc[train["primary_label"].isin(missing_birds)].sample(frac=1).drop_duplicates("primary_label").index.to_list()
        print(f"Adding next filenames:\n", train["filename"].iloc[index_to_add])
        split[fold_i][0] = np.concatenate([split[fold_i][0], np.array(index_to_add)])
split = [tuple(el) for el in split]      

In [ ]:
for train_idx, _ in split:
    assert set(train["primary_label"].iloc[split[fold_i][0]]) == all_unique_birds

In [ ]:
assert list(train.index) == list(range(len(train)))

In [ ]:
train.loc[train["primary_label"].isin(missing_birds)].sample(frac=1).drop_duplicates("primary_label").index.to_list()

In [ ]:
train["group"]

In [ ]:
train["group"].str.lower().value_counts().to_dict()

# Check CV Splits

In [39]:
df = pd.read_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur.csv"
)
cv_split = np.load(
    "../data/cv_split_base_and_prev_comps_XCsnipet28032025_csa_group_foldinter.npy", allow_pickle=True
)

/tmp/ipykernel_1079241/1162802926.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [40]:
df["filename"].value_counts()

filename
1139490/CSA36385.ogg    1
y00678/XC456643.ogg     1
y00678/XC428810.ogg     1
y00678/XC429779.ogg     1
y00678/XC430769.ogg     1
                       ..
linwoo1/XC550450.ogg    1
linwoo1/XC565143.ogg    1
linwoo1/XC565171.ogg    1
linwoo1/XC569282.ogg    1
24292/116126.wav        1
Name: count, Length: 39204, dtype: int64

In [41]:
for fold_id, (train_ids, val_ids) in enumerate(cv_split):
    train_fold, val_fold = df.iloc[train_ids], df.iloc[val_ids]
    assert not set(train_fold["filename"]) & set(val_fold["filename"])
    assert set(train_ids) == set(np.concatenate([val_ids for _, val_ids in np.delete(cv_split, fold_id, 0)]))
    # assert not set(train_fold["author"]) & set(val_fold["author"])

AssertionError: 

In [42]:
for fold_id, (train_ids, val_ids) in enumerate(cv_split):
    print("Len Val", len(val_ids), "Len Train", len(train_ids), "Len DF", len(df), "Len Val + Train", len(val_ids) + len(train_ids))
    train_fold, val_fold = df.iloc[train_ids], df.iloc[val_ids]
    print("Intresect Len:", len(set(train_fold["filename"]) & set(val_fold["filename"])))
    assert set(train_fold["filename"]) | set(val_fold["filename"]) == set(df["filename"])
    assert set(train_fold["primary_label"]) == set(df["primary_label"])
    assert set(val_fold["primary_label"]) == set(df["primary_label"])
    assert set(train_ids) == set(np.concatenate([val_ids for _, val_ids in np.delete(cv_split, fold_id, 0)]))
    # assert not set(train_fold["author"]) & set(val_fold["author"])

Len Val 8467 Len Train 30856 Len DF 39204 Len Val + Train 39323
Intresect Len: 35
Len Val 7979 Len Train 31344 Len DF 39204 Len Val + Train 39323
Intresect Len: 37
Len Val 8903 Len Train 30420 Len DF 39204 Len Val + Train 39323
Intresect Len: 38
Len Val 7358 Len Train 31965 Len DF 39204 Len Val + Train 39323
Intresect Len: 51
Len Val 6616 Len Train 32707 Len DF 39204 Len Val + Train 39323
Intresect Len: 50
